<a href="https://colab.research.google.com/github/emmanuelnh17/Proyecto-Sustituto/blob/main/fase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Competencia Titanic con TensorFlow Decision Forests

#Este cuaderno te guiará a través de los pasos necesarios para entrenar un modelo base de Gradient Boosted Trees usando TensorFlow Decision Forests y crear una presentación para la competencia Titanic.

#Este cuaderno muestra:

#1. Cómo realizar un procesamiento básico de datos. Por ejemplo, se tokenizarán los nombres de los pasajeros y se dividirán los nombres de los tickets en partes.
#2. Cómo entrenar un modelo de Gradient Boosted Trees (GBT) con parámetros predeterminados.
#3. Cómo entrenar un GBT con parámetros predeterminados mejorados.
#4. Cómo ajustar los parámetros de un GBT.
#5. Cómo entrenar y combinar muchos GBTs.

#Codigo copiado de

#Gusthema (Owner)

#achoum (Editor)


In [8]:
# Importa las librerías necesarias para manejar archivos, descargar y extraer datos
import os
import sys
from tempfile import NamedTemporaryFile  # Para crear archivos temporales
from urllib.request import urlopen  # Para descargar archivos desde una URL
from urllib.parse import unquote, urlparse  # Para manejar URLs
from urllib.error import HTTPError  # Para manejar errores HTTP
from zipfile import ZipFile  # Para manejar archivos ZIP
import tarfile  # Para manejar archivos TAR
import shutil  # Para eliminar directorios

# Constante que define el tamaño de los fragmentos de descarga
CHUNK_SIZE = 40960  # Tamaño del fragmento que se descarga en cada iteración (40 KB)
# Mapeo de la fuente de datos del Titanic, contiene la URL de descarga codificada
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240908%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240908T222413Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db7bf4cf8ca0e5bff88fe0a1f81b5c9e726e6aeceebd9d79c210062cc4e63faeec513a70ff1fe94c755078806ae9518e41f121cfedb051c8698d94fa2f30961fef7b2d00919643020416e2ab88a503bfa0bf5e25740fc05b13fc06294ce3ce3735801b56ebce0836536a4a8abd5b54fd8ef7ec0702b8bc7dbd14fbc1f7612296a6043c6f959780fff4a0fd53e7e6c8c98c7d15d1d1c94f5abe953bee4a9f1dd583d8cf8e9ce3c88afdd062f904c44638dee54590ed21c4a4d2bed31b7aa0164c051511ae35c9be615b3ab61962c6b741affb52fa23e787d9a42ff48a3c17dbd990085dd9f1c9df632b3294edf99232b651842fe445878d03af38b777cbb733472'

# Definir rutas para los directorios de trabajo y entrada
KAGGLE_INPUT_PATH = '/kaggle/input'  # Ruta donde se colocarán los datos de entrada
KAGGLE_WORKING_PATH = '/kaggle/working'  # Ruta de trabajo para archivos temporales y resultados
KAGGLE_SYMLINK = 'kaggle'  # Nombre del enlace simbólico que se crea para facilitar el acceso

# Limpiar y preparar los directorios de trabajo
!umount /kaggle/input/ 2> /dev/null  # Desmonta el directorio de entrada si está montado
shutil.rmtree('/kaggle/input', ignore_errors=True)  # Elimina el directorio de entrada si existe
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)  # Crea el directorio de entrada con permisos
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)  # Crea el directorio de trabajo con permisos

# Crear enlaces simbólicos para los directorios de entrada y trabajo si no existen
try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass  # Ignora el error si el enlace ya existe

try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass  # Ignora el error si el enlace ya existe

# Procesar la fuente de datos, descargar y extraer los archivos
for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')  # Divide el mapeo en directorio y URL codificada
    download_url = unquote(download_url_encoded)  # Decodifica la URL
    filename = urlparse(download_url).path  # Extrae el nombre del archivo de la URL
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)  # Ruta donde se guardará el archivo descomprimido

    try:
        # Descargar el archivo y guardarlo temporalmente
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']  # Obtiene el tamaño total del archivo
            print(f'Descargando {directory}, {total_length} bytes comprimidos')  # Imprime el tamaño del archivo

            # Descargar el archivo en fragmentos
            dl = 0  # Inicializa el contador de bytes descargados
            data = fileres.read(CHUNK_SIZE)  # Lee el primer fragmento
            while len(data) > 0:
                dl += len(data)  # Suma la cantidad de bytes descargados
                tfile.write(data)  # Escribe el fragmento en el archivo temporal
                done = int(50 * dl / int(total_length))  # Calcula el progreso de la descarga
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes descargados")  # Imprime el progreso
                sys.stdout.flush()  # Asegura que se imprima en tiempo real
                data = fileres.read(CHUNK_SIZE)  # Lee el siguiente fragmento

            # Descomprimir el archivo según su formato (ZIP o TAR)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)  # Extrae el archivo ZIP
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)  # Extrae el archivo TAR

            print(f'\nDescargado y descomprimido: {directory}')  # Imprime mensaje de éxito
    except HTTPError as e:
        # Manejo de errores HTTP (como URL expiradas)
        print(f'Error al cargar (posiblemente caducado) {download_url} en la ruta {destination_path}')
        continue  # Continúa con la siguiente URL
    except OSError as e:
        # Manejo de otros errores de archivos
        print(f'Error al cargar {download_url} en la ruta {destination_path}')
        continue  # Continúa con la siguiente URL

print('Importación de fuentes de datos completada.')  # Imprime mensaje final de éxito


Descargando titanic, 34877 bytes comprimidos
[==================================================] 34877 bytes descargados
Descargado y descomprimido: titanic
Importación de fuentes de datos completada.


In [2]:
# Instala la librería TensorFlow Decision Forests.
# Esta librería permite trabajar con modelos de aprendizaje automático basados en árboles de decisión.
!pip install tensorflow_decision_forests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 35.1 MB/s eta 0:00:00


# Importar las dependencias


In [9]:
# Importa la librería NumPy para realizar operaciones matemáticas y manipulación de matrices
import numpy as np

# Importa la librería pandas para el manejo y análisis de estructuras de datos (DataFrames)
import pandas as pd

# Importa la librería os para interactuar con el sistema operativo, como acceder a directorios y archivos
import os

# Importa la librería TensorFlow, que es utilizada para construir y entrenar modelos de Machine Learning
import tensorflow as tf

# Importa TensorFlow Decision Forests, una librería especializada en modelos basados en bosques aleatorios y árboles de decisión
import tensorflow_decision_forests as tfdf

# Imprime la versión de TensorFlow Decision Forests instalada, para asegurar que se está utilizando la correcta
print(f"Se encontró TF-DF versión {tfdf.__version__}")


Se encontró TF-DF versión 1.10.0


# Cargar el conjunto de datos


In [10]:
# Carga el conjunto de datos de entrenamiento del archivo CSV ubicado en la carpeta de entrada.
# train.csv contiene los datos históricos de los pasajeros del Titanic (características y si sobrevivieron o no).
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")

# Carga el conjunto de datos de prueba (serving_df), que no incluye la columna de "Supervivencia" (Survived).
# test.csv se utiliza para hacer predicciones basadas en las características de los pasajeros.
serving_df = pd.read_csv("/kaggle/input/titanic/test.csv")

# Muestra las primeras 10 filas del conjunto de datos de entrenamiento para tener una vista rápida de la estructura y contenido.
train_df.head(10)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


# Preparar el conjunto de datos

Aplicaremos las siguientes transformaciones al conjunto de datos:

1. Tokenizar los nombres. Por ejemplo, "Braund, Mr. Owen Harris" se convertirá en ["Braund", "Mr.", "Owen", "Harris"].
2. Extraer el prefijo del ticket. Por ejemplo, el ticket "STON/O2. 3101282" se convertirá en "STON/O2." y 3101282.


In [11]:
# Define una función para preprocesar el conjunto de datos.
def preprocess(df):
    # Crea una copia del DataFrame para evitar modificar el original directamente.
    df = df.copy()

    # Normaliza los nombres eliminando caracteres especiales como comas, paréntesis, corchetes y comillas.
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])

    # Extrae el número de boleto, que se encuentra al final del string en la columna 'Ticket'.
    def ticket_number(x):
        return x.split(" ")[-1]

    # Extrae los componentes del boleto, eliminando el número (último elemento).
    # Si no hay componentes adicionales, se asigna "NONE".
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])

    # Aplica la normalización de nombres en la columna 'Name'.
    df["Name"] = df["Name"].apply(normalize_name)

    # Crea una nueva columna 'Ticket_number' con el número del boleto extraído.
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)

    # Crea una nueva columna 'Ticket_item' con el ítem del boleto (sin el número final).
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)

    # Devuelve el DataFrame preprocesado.
    return df

# Aplica la función de preprocesamiento al conjunto de datos de entrenamiento.
preprocessed_train_df = preprocess(train_df)

# Aplica la función de preprocesamiento al conjunto de datos de prueba (serving_df).
preprocessed_serving_df = preprocess(serving_df)

# Muestra las primeras 5 filas del conjunto de datos de entrenamiento preprocesado.
preprocessed_train_df.head(5)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


 Mantenemos la lista de características de entrada del modelo. En particular, no queremos entrenar nuestro modelo con las características "PassengerId" y "Ticket".


In [12]:
# Crea una lista de las columnas (características) del conjunto de datos preprocesado.
input_features = list(preprocessed_train_df.columns)

# Elimina la columna 'Ticket', ya que no se utilizará directamente como característica para el modelo.
input_features.remove("Ticket")

# Elimina la columna 'PassengerId', ya que es un identificador único que no aporta valor predictivo.
input_features.remove("PassengerId")

# Elimina la columna 'Survived', ya que es la etiqueta objetivo (lo que queremos predecir).
input_features.remove("Survived")

# La columna 'Ticket_number' podría ser eliminada si no se desea incluirla, aquí está comentada.
# Se podría descomentar si se quiere excluirla como característica.
# input_features.remove("Ticket_number")

# Imprime las características de entrada seleccionadas que serán utilizadas por el modelo.
print(f"Características de entrada: {input_features}")


Características de entrada: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


# Convertir el conjunto de datos de Pandas a un conjunto de datos de TensorFlow


In [13]:
# Define una función para dividir los nombres en tokens. TensorFlow Decision Forests (TF-DF) puede manejar tokens de texto nativamente.
def tokenize_names(features, labels=None):
    """Divide los nombres en tokens. TF-DF puede consumir tokens de texto de manera nativa."""
    features["Name"] = tf.strings.split(features["Name"])  # Divide los nombres en tokens usando espacios como separadores
    return features, labels

# Convierte el DataFrame preprocesado de entrenamiento a un conjunto de datos de TensorFlow.
# Especifica la columna 'Survived' como la etiqueta (variable objetivo) y aplica la función de tokenización.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df, label="Survived").map(tokenize_names)

# Convierte el DataFrame preprocesado de prueba a un conjunto de datos de TensorFlow.
# No se especifica una etiqueta, ya que este conjunto de datos se utiliza para hacer predicciones.
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)


# Entrenar el modelo con parámetros predeterminados

### Entrenar el modelo

Primero, entrenaremos un modelo de GradientBoostedTreesModel con los parámetros predeterminados.


In [14]:
# Define un modelo de Gradient Boosted Trees usando TensorFlow Decision Forests.
# Configura el modelo con los siguientes parámetros:
# - verbose=0: Muestra muy pocos logs durante el entrenamiento.
# - features: Lista de características especificadas para el modelo (excluye cualquier característica no especificada).
# - exclude_non_specified_features=True: Usa solo las características en la lista "features".
# - random_seed=1234: Establece una semilla para garantizar la reproducibilidad de los resultados.
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Muy pocos logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Usa solo las características en "features"
    random_seed=1234,
)

# Entrena el modelo usando el conjunto de datos de entrenamiento.
model.fit(train_ds)

# Obtiene el inspector del modelo para evaluar el rendimiento.
self_evaluation = model.make_inspector().evaluation()

# Imprime la precisión y la pérdida del modelo en el conjunto de entrenamiento.
# La precisión mide la exactitud de las predicciones, mientras que la pérdida evalúa el error del modelo.
print(f"Precisión: {self_evaluation.accuracy} Pérdida: {self_evaluation.loss}")


Precisión: 0.8260869383811951 Pérdida: 0.8608942627906799


# Entrenar el modelo con parámetros predeterminados mejorados

Ahora usaremos algunos parámetros específicos al crear el modelo GBT.


In [15]:
# Define un modelo de Gradient Boosted Trees usando TensorFlow Decision Forests con parámetros adicionales.
# Configura el modelo con los siguientes parámetros:
# - verbose=0: Muestra muy pocos logs durante el entrenamiento.
# - features: Lista de características especificadas para el modelo (usa solo las características en "features").
# - exclude_non_specified_features=True: Usa solo las características en la lista "features".
# - min_examples=1: Número mínimo de ejemplos necesarios para realizar una división en el árbol.
# - categorical_algorithm="RANDOM": Algoritmo utilizado para manejar variables categóricas (aleatorio en este caso).
# - shrinkage=0.05: Tasa de reducción del gradiente (shrinkage) para mejorar el rendimiento del modelo.
# - split_axis="SPARSE_OBLIQUE": Tipo de partición utilizada para dividir las características en árboles oblicuos dispersos.
# - sparse_oblique_normalization="MIN_MAX": Normalización aplicada a las características oblicuas dispersas.
# - sparse_oblique_num_projections_exponent=2.0: Exponente utilizado para el número de proyecciones oblicuas dispersas.
# - num_trees=2000: Número de árboles en el modelo.
# - random_seed=1234: Establece una semilla para garantizar la reproducibilidad de los resultados.

model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Muy pocos logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Usa solo las características en "features"
    min_examples=1,
    categorical_algorithm="RANDOM",
    shrinkage=0.05,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=2000,
    random_seed=1234,
)

# Entrena el modelo utilizando el conjunto de datos de entrenamiento.
model.fit(train_ds)

# Obtiene el inspector del modelo para evaluar su rendimiento en el conjunto de entrenamiento.
self_evaluation = model.make_inspector().evaluation()

# Imprime la precisión y la pérdida del modelo en el conjunto de entrenamiento.
# La precisión indica la exactitud de las predicciones del modelo, mientras que la pérdida evalúa el error del modelo.
print(f"Precisión: {self_evaluation.accuracy} Pérdida: {self_evaluation.loss}")


Precisión: 0.75 Pérdida: 1.0462466478347778


Veamos el modelo y también se puede observar la información sobre la importancia de las variables que el modelo ha determinado


In [16]:
# Muestra un resumen del modelo de Gradient Boosted Trees.
# El resumen proporciona información sobre la estructura del modelo, incluyendo detalles
# sobre el número de árboles, las características utilizadas y otros parámetros del modelo.
model.summary()


Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.805245 ################
    2.           "Age"  0.372369 #####
    3.          "Fare"  0.273026 ##
    4.          "Name"  0.187307 
    5.        "Pclass"  0.180805 
    6.   "Ticket_item"  0.178416 
    7. "Ticket_number"  0.178366 
    8.         "Parch"  0.177684 
    9.      "Embarked"  0.176070 
   10.         "SibSp"  0.172557 

Variable Importance: NUM_AS_ROOT:
    1.  "Sex" 34.000000 ##########

# Realizar predicciones


In [17]:
# Define una función para convertir las predicciones del modelo al formato requerido por Kaggle.
# La función toma las probabilidades de supervivencia y las convierte en una DataFrame con dos columnas:
# - "PassengerId": Identificador del pasajero.
# - "Survived": Resultado binario (0 o 1) basado en el umbral especificado (por defecto 0.5).
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(serving_ds, verbose=0)[:,0]  # Predice las probabilidades de supervivencia para el conjunto de prueba
    return pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],  # Incluye el identificador del pasajero
        "Survived": (proba_survive >= threshold).astype(int)  # Convierte las probabilidades en etiquetas binarias usando el umbral
    })

# Define una función para guardar las predicciones en un archivo CSV en el formato esperado por Kaggle.
# La función guarda el DataFrame en la ruta especificada y muestra un mensaje confirmando la exportación.
def make_submission(kaggle_predictions):
    path="/kaggle/working/submission.csv"  # Ruta donde se guardará el archivo CSV
    kaggle_predictions.to_csv(path, index=False)  # Guarda el DataFrame como archivo CSV sin el índice
    print(f"Submission exported to {path}")  # Muestra un mensaje con la ruta del archivo exportado

# Genera las predicciones usando el modelo y las convierte al formato requerido por Kaggle.
kaggle_predictions = prediction_to_kaggle_format(model)

# Crea el archivo de envío con las predicciones en el formato adecuado y lo guarda en la ruta especificada.
make_submission(kaggle_predictions)

# Muestra las primeras líneas del archivo CSV para verificar el contenido.
!head /kaggle/working/submission.csv


Submission exported to /kaggle/working/submission.csv
PassengerId,Survived
892,0
893,0
894,0
895,0
896,0
897,1
898,0
899,0
900,1


# Entrenamiento de un modelo con ajuste de hiperparámetros

El ajuste de hiperparámetros se activa especificando el argumento del constructor `tuner` del modelo. El objeto `tuner` contiene toda la configuración del ajuste (espacio de búsqueda, optimizador, ensayo y objetivo).



In [20]:
# Configura un proceso de búsqueda aleatoria para la optimización de hiperparámetros del modelo.
# Utiliza la búsqueda aleatoria para explorar diferentes combinaciones de parámetros.

# Define el objeto `tuner` para la búsqueda aleatoria con un número máximo de 1000 pruebas.
tuner = tfdf.tuner.RandomSearch(num_trials=1000)

# Define el espacio de búsqueda para los hiperparámetros específicos.
# Selecciona entre diferentes valores para el número mínimo de ejemplos necesarios para realizar una división.
tuner.choice("min_examples", [2, 5, 7, 10])

# Selecciona el algoritmo para manejar variables categóricas.
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

# Configura el espacio de búsqueda para la estrategia de crecimiento de árboles.
# Permite ajustar el máximo de profundidad para la estrategia de crecimiento LOCAL.
local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

# Configura el espacio de búsqueda para la estrategia de crecimiento GLOBAL.
# Permite ajustar el número máximo de nodos para la estrategia BEST_FIRST_GLOBAL.
global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

# Opcionalmente, se puede ajustar el uso de la ganancia Hessiana.
# tuner.choice("use_hessian_gain", [True, False])

# Ajusta la tasa de reducción del gradiente (shrinkage).
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])

# Ajusta el ratio de atributos candidatos para la partición de características.
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

# Ajusta el eje de partición de características.
tuner.choice("split_axis", ["AXIS_ALIGNED"])

# Configura el espacio de búsqueda para el caso de particiones oblicuas dispersas.
# Permite ajustar la normalización, el tipo de pesos y el número de proyecciones oblicuas.
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

# Ajusta el modelo utilizando el proceso de búsqueda aleatoria definido.
# El modelo se entrena con el conjunto de datos de entrenamiento.
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)

# Evalúa el rendimiento del modelo ajustado usando el conjunto de datos de entrenamiento.
tuned_self_evaluation = tuned_model.make_inspector().evaluation()

# Imprime la precisión y la pérdida del modelo ajustado en el conjunto de entrenamiento.
print(f"Precisión: {tuned_self_evaluation.accuracy} Pérdida:{tuned_self_evaluation.loss}")


Use /tmp/tmpaql70hqp as temporary training directory
Precisión: 0.8630136847496033 Pérdida:0.6749962568283081


En la última línea de la celda anterior, se puede ver que la precisión es mayor que con los parámetros predeterminados y los parámetros establecidos manualmente.

Esta es la idea principal detrás del ajuste de hiperparámetros.

Para más información, puedes seguir este tutorial: [Ajuste automático de hiperparámetros](https://www.tensorflow.org/decision_forests/tutorials/automatic_tuning_colab)


# Crear un ensamblaje

Aquí se creara 100 modelos con diferentes semillas y se combinaran sus resultados.

Este enfoque reduce un poco los aspectos aleatorios relacionados con la creación de modelos de aprendizaje automático.

En la creación de GBT se utiliza el parámetro `honest`. Este parámetro usa diferentes ejemplos de entrenamiento para inferir la estructura y los valores de las hojas. Esta técnica de regularización intercambia ejemplos por estimaciones de sesgo.


In [21]:
# Inicializa variables para almacenar las predicciones y contar el número de modelos entrenados.
predictions = None
num_predictions = 0

# Realiza la evaluación con 100 modelos diferentes.
for i in range(100):
    print(f"i:{i}")  # Imprime el índice de la iteración actual

    # Crea un modelo de Gradient Boosted Trees con parámetros definidos.
    # La semilla aleatoria (random_seed) cambia en cada iteración para obtener diferentes modelos.
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0,  # Muestra pocos logs durante el entrenamiento
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True,  # Usa solo las características especificadas en "features"
        random_seed=i,  # Cambia la semilla aleatoria en cada iteración para obtener diferentes resultados
        honest=True,  # Activa el modo honesto para una evaluación más precisa
    )
    # Entrena el modelo usando el conjunto de datos de entrenamiento.
    model.fit(train_ds)

    # Realiza predicciones sobre el conjunto de datos de prueba.
    sub_predictions = model.predict(serving_ds, verbose=0)[:,0]

    # Agrega las predicciones actuales a las predicciones acumuladas.
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions

    # Incrementa el contador de modelos entrenados.
    num_predictions += 1

# Calcula el promedio de las predicciones de todos los modelos.
predictions /= num_predictions

# Convierte las predicciones promedio a un DataFrame en el formato requerido por Kaggle.
kaggle_predictions = pd.DataFrame({
    "PassengerId": serving_df["PassengerId"],  # Incluye el identificador del pasajero
    "Survived": (predictions >= 0.5).astype(int)  # Convierte las probabilidades en etiquetas binarias usando el umbral 0.5
})

# Guarda las predicciones en un archivo CSV y muestra un mensaje con la ruta del archivo.
make_submission(kaggle_predictions)


i:0


i:1


i:2


i:3
i:4
i:5
i:6
i:7
i:8
i:9
i:10
i:11
i:12
i:13
i:14
i:15
i:16
i:17
i:18
i:19
i:20
i:21
i:22
i:23
i:24
i:25
i:26
i:27
i:28
i:29
i:30
i:31
i:32
i:33
i:34
i:35
i:36
i:37
i:38
i:39
i:40
i:41
i:42
i:43
i:44
i:45
i:46
i:47
i:48
i:49
i:50
i:51
i:52
i:53
i:54
i:55
i:56
i:57
i:58
i:59
i:60
i:61
i:62
i:63
i:64
i:65
i:66
i:67
i:68
i:69
i:70
i:71
i:72
i:73
i:74
i:75
i:76
i:77
i:78
i:79
i:80
i:81
i:82
i:83
i:84
i:85
i:86
i:87
i:88
i:89
i:90
i:91
i:92
i:93
i:94
i:95
i:96
i:97
i:98
i:99
Submission exported to /kaggle/working/submission.csv
